In [1]:
######### loading the packages
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
import matplotlib.patches as patches
import ipywidgets as widgets
import voila
from IPython.display import display, HTML

In [39]:
# Instead of a global directory, just use relative folders:
# os.chdir("/Users/ruofeiguo/CEO Compensation/my-timeline-repo")
data_directory = "data"
# graph_directory = "graph"

In [3]:
base = pd.read_csv(os.path.join(data_directory, 'SumComp.csv'))
base['CIK'] = base['CIK'].astype(str).str.replace(r"[^\d]", "", regex=True).astype(int)
# # base.to_csv(os.path.join(data_directory, 'SumComp.csv'), index=False)

In [4]:
grant = pd.read_csv(os.path.join(data_directory, 'GpbaGrant_clean.csv'))
# grant = grant[['grantID', 'performanceGrouping']]
# grant.to_csv(os.path.join(data_directory, 'GpbaGrant_clean.csv'), index=False)
# print(grant.head)

In [5]:
time = pd.read_csv(os.path.join(data_directory, 'GpbaTime_vest.csv'))

In [52]:
# df_comp = pd.read_csv(os.path.join(data_directory, 'GpbaAbs_vest_clean.csv'), encoding="latin1")
# print(df_comp.head)

# def resolve_companyname_cik_conflicts(df_comp):
#     # Step 1: Resolve multiple companyNames for the same CIK
#     df_comp_sorted = df_comp.sort_values(by=['CIK', 'fiscalYear', 'companyName'], ascending=[True, False, False])

#     # Select the best companyName for each CIK (latest fiscalYear, longest name)
#     best_company_mapping = df_comp_sorted.drop_duplicates(subset=['CIK'], keep='first').set_index('CIK')['companyName']

#     # Map the best companyName to all rows with that CIK
#     df_comp['companyName'] = df_comp['CIK'].map(best_company_mapping)

#     # Step 2: Resolve multiple CIKs for the same companyName
#     cik_counts = df_comp.groupby('companyName')['CIK'].transform('nunique')  # Unique CIKs per company
#     multiple_cik_mask = cik_counts > 1  # Identify companies linked to multiple CIKs

#     # Assign ranks to CIKs within the same company
#     df_comp['cik_rank'] = df_comp.groupby('companyName')['CIK'].rank(method='dense', ascending=True).astype(int)

#     # Rename only when needed
#     df_comp.loc[multiple_cik_mask, 'companyName'] = df_comp['companyName'] + "(" + df_comp['cik_rank'].astype(str) + ")"

#     # Drop the helper column
#     df_comp.drop(columns=['cik_rank'], inplace=True)

#     # Step 3: Verify One-to-One Mapping
#     cik_to_name_counts = df_comp.groupby('CIK')['companyName'].nunique()
#     name_to_cik_counts = df_comp.groupby('companyName')['CIK'].nunique()

#     assert cik_to_name_counts.max() == 1, "Some CIKs still map to multiple companyNames!"
#     assert name_to_cik_counts.max() == 1, "Some companyNames still map to multiple CIKs!"
    
#     # Ensure the shape remains the same
#     assert df_comp.shape[0] == original_shape, "Row count changed! No rows should be dropped."

#     return df_comp

# # Apply the function
# original_shape = df_comp.shape[0]
# df_final = resolve_companyname_cik_conflicts(df_comp)

# # Display the final DataFrame
# print(df_final.head)
# df_final.to_csv(os.path.join(data_directory, 'GpbaAbs_vest_clean.csv'), index=False)

<bound method NDFrame.head of         fiscalYear  grantId            metric       size      CIK  \
0             2006       32          Earnings   999375.0   884887   
1             2006       32        Individual   333125.0   884887   
2             2007       49          Earnings   400000.0  1134538   
3             2007       51  Operating Income  2999982.0  1134538   
4             2007       53          Earnings   450000.0  1134538   
...            ...      ...               ...        ...      ...   
156946        2022  2149857    Gross Revenues  1183845.6  1571996   
156947        2020  2150892               EPS   579637.5   937556   
156948        2020  2150892    Gross Revenues   579637.5   937556   
156949        2020  2150897     Profit Margin  4499943.5   937556   
156950        2020  2150897    Gross Revenues  4499943.5   937556   

        participantid   grantDate   startDate     endDate  \
0             33184.0  2006-01-01  2006-01-01  2007-01-01   
1             33184

In [53]:
df_comp = pd.read_csv(os.path.join(data_directory, 'GpbaAbs_vest_clean.csv'), encoding="latin1")
df_comp['participantid'] = df_comp['participantid'].astype(int)

unique_list = (
    df_comp[['CIK', 'companyName', 'participantid']]
    .drop_duplicates()
    .values
    .tolist()
)

unique_cik = (
    df_comp[['CIK', 'companyName']]
    .drop_duplicates()
    .values
    .tolist()
)
cik_to_company = {cik: name for cik, name in unique_cik}
company_to_cik = {name: cik for cik, name in unique_cik}

unique_ceo = (
    df_comp[['companyName', 'participantid']]
    .drop_duplicates()
    .values
    .tolist()
)

In [54]:
# df_rel = pd.read_csv(os.path.join(data_directory, 'GpbaRel_vest.csv'))
# df_rel['companyName'] = df_rel['CIK'].map(cik_to_company)
# df_rel.to_csv(os.path.join(data_directory, 'GpbaRel_vest.csv'), index=False)

In [33]:
def create_df_combined(company_name=None, participantid=None, dis=False):
    """
    Reads the CSV files, filters on company_name (substring match, case-insensitive),
    CIK, and participantid if they are provided. Returns the combined DataFrame with
    columns: [CIK, companyName, metric, fiscalYear, grantDate, startDate, endDate, value, absRel].
    """

    # --- Read CSVs ---
    df_abs = pd.read_csv(os.path.join(data_directory, 'GpbaAbs_vest_clean.csv'))
    df_rel = pd.read_csv(os.path.join(data_directory, 'GpbaRel_vest.csv'))

    #### get CIK from unique list
    CIK = company_to_cik[company_name]

    # --- Filter the DataFrames ---
    df_abs_filtered = df_abs[(df_abs['CIK'] == CIK) & (df_abs['participantid'] == float(participantid))]
    df_rel_filtered = df_rel[(df_rel['CIK'] == CIK) & (df_rel['participantid'] == float(participantid))]

    if df_abs_filtered.empty and df_rel_filtered.empty:
        print(f"No data found for CIK={CIK}, participant={participantid}.")
        return  # nothing else to plot
    
    # --- Select columns and label abs/rel ---
    columns_to_display = [
        'CIK', 'companyName', 'metric', 'fiscalYear', 
        'grantDate', 'startDate', 'endDate', 'size', 'percentVest', 'value', 'grantId', 'group'
    ]
    df_abs_filtered = df_abs_filtered[columns_to_display].copy()
    df_rel_filtered = df_rel_filtered[columns_to_display].copy()

    df_abs_filtered['absRel'] = 'abs'
    df_rel_filtered['absRel'] = 'rel'

    df_combined = pd.concat([df_abs_filtered, df_rel_filtered], ignore_index=True)
    df_combined = df_combined.sort_values(by=['metric', 'grantDate'], ascending=[True, True])

    mask = (
        df_combined['percentVest'].isna() &
        df_combined['size'].isna() &
        (df_combined['metric'] == 'Unknown') &
        (df_combined['startDate'] == df_combined['grantDate']) &
        (df_combined['grantDate'] == df_combined['endDate'])
    )
    df_combined = df_combined[~mask]


    # Display the combined DataFrame (in notebook)
    if dis:
        pd.set_option('display.max_rows', None)
        pd.set_option('display.max_columns', None)
        display(df_combined)

    return df_combined

In [56]:
def plot_compensation_timeline_by_grant(company_name, participantid, dis=False):
    grant_intro = 'Information on metrics and their groupings:\n'
    """
    Group df_combined by 'grantId'. 
    For each group, determine distinct (grantDate, startDate, endDate) combos.
    Plot one horizontal line per distinct combo. 
    Label the y-axis with:
      - if there's only one distinct timeline, show the list of metrics in the group
      - if multiple timelines exist, show separate lines, labeling each line with the relevant metrics
    """
    df_combined = create_df_combined(company_name=company_name, participantid=participantid, dis=dis)
    if df_combined.empty:
        print(f"No data found for company={company_name}, CIK={CIK}, participant={participantid}.")
        return

    # Just in case, ensure these are datetimes
    df_combined['grantDate'] = pd.to_datetime(df_combined['grantDate'], errors='coerce')
    df_combined['startDate'] = pd.to_datetime(df_combined['startDate'], errors='coerce')
    df_combined['endDate']   = pd.to_datetime(df_combined['endDate'], errors='coerce')

    CIK = company_to_cik[company_name]
    
    # --- Retrieve base salary data ---
    if pd.isna(CIK):  # If company_name is used, map it to CIK
        CIK = df_combined['CIK'].iloc[0] if not df_combined.empty else None

    base_filtered = base[(base['CIK'] == CIK) & (base['participantid'] == int(participantid))]
    time_filtered = time[(time['CIK'] == CIK) & (time['participantid'] == int(participantid))]

    # Convert fiscalYear to integer for sorting
    base_filtered.loc[:,'FiscalYear'] = pd.to_numeric(base_filtered['FiscalYear'], errors='coerce')
    
    # --- Compute the minimum value for scaling line thickness ---
    max_value = pd.concat([df_combined['size'], base_filtered['salary'], base_filtered['stockAwards'], base_filtered['optionAwards'], time_filtered['grantDateFV']], axis=0).dropna().max()

    # Group by grantId
    grouped = df_combined.groupby('grantId', dropna=False)
    group_grant_dates = {grantId: sub_df['grantDate'].min() for grantId, sub_df in grouped}
    sorted_groups = sorted(group_grant_dates.items(), key=lambda x: x[1])

    # Prepare the figure
    fig_width = 12  # Keep a fixed width
    fig_height = max(6, (len(grouped)+1) * 0.5)  # Scale height based on number of items
    fig, ax = plt.subplots(figsize=(fig_width, fig_height))

    # For coloring, reuse your color scheme if you want
    # palette = sns.color_palette("colorblind", 10)
    color_map = {'abs': '#4C72B0', 'rel': '#F4A261'}
    
    # We'll keep track of the y-position for each group
    current_y = 10
    y_positions = []
    y_labels = []

    # Track legend entries
    legend_labels = {
        'absolute, performance period': False,
        'relative, performance period': False,
        'Grant Date': False,
        'Non-performance-based': False,
        'Time-based': False
    }

    # --- Plot Base Salary ---
    max_salary_award = 0 if base_filtered['salary'].dropna().empty else max(20 * base_filtered['salary'].dropna().max() / max_value, 0.3)
    max_stock_award = 0 if base_filtered['stockAwards'].dropna().empty else max(20 * base_filtered['stockAwards'].dropna().max() / max_value, 0.3)
    max_option_award = 0 if base_filtered['optionAwards'].dropna().empty else max(20 * base_filtered['optionAwards'].dropna().max() / max_value, 0.3)
    base_salary_y = current_y
    stock_y = base_salary_y + max_salary_award/2 + 4 + max_stock_award/2
    option_y = stock_y + max_stock_award/2 + 4 + max_option_award/2
    ax.axhline(y=base_salary_y + max_salary_award/2 + 2, color='gray', linestyle='dashed', alpha=0.3)
    ax.axhline(y=stock_y + max_stock_award/2 + 2, color='gray', linestyle='dashed', alpha=0.3)
    for _, row in base_filtered.iterrows():
        fiscal_year = int(row['FiscalYear'])
        salary_value = row['salary']
        salary_height = 0
        if pd.notna(salary_value):
            height = max(20 * salary_value / max_value, 1)
            # Create a rectangle for base salary
            base_salary_rect = patches.Rectangle(
                (pd.Timestamp(f"{fiscal_year}-01-01"), base_salary_y - height / 2),  # (x, y) position
                pd.Timedelta(days=365),  # Width covering the full year
                height,  # Height scaled to salary value
                color='black',
                label='Non-performance-based'
            )
            
            ax.add_patch(base_salary_rect)  # Add rectangle to the plot

            legend_labels['Non-performance-based'] = True

        stock_value = row['stockAwards']
        if pd.notna(stock_value):
            height = max(20 * stock_value / max_value, 1)
            # Create a rectangle for base salary
            stock_rect = patches.Rectangle(
                (pd.Timestamp(f"{fiscal_year}-01-01"), stock_y - height / 2),  # (x, y) position
                pd.Timedelta(days=365),  # Width covering the full year
                height,  # Height scaled to salary value
                color='black',
                label='Non-performance-based'
            )
            
            ax.add_patch(stock_rect)  # Add rectangle to the plot
            legend_labels['Non-performance-based'] = True

        option_value = row['optionAwards']
        if pd.notna(option_value):
            height = max(20 * option_value / max_value, 1)
            # Create a rectangle for base salary
            option_rect = patches.Rectangle(
                (pd.Timestamp(f"{fiscal_year}-01-01"), option_y - height / 2),  # (x, y) position
                pd.Timedelta(days=365),  # Width covering the full year
                height,  # Height scaled to salary value
                color='black',
                label='Non-performance-based'
            )
            
            ax.add_patch(option_rect)  # Add rectangle to the plot
            legend_labels['Non-performance-based'] = True

    current_y = option_y + max_option_award/2 + 2
    # --- Plot Time-Based ---
    ax.axhline(y=current_y, color='gray', linestyle='dashed', alpha=0.3)
    # print(time_filtered)
    time_filtered = time_filtered.sort_values(by='grantDate')
    time_y = current_y
    current_y += 2
    all_nan = time_filtered['grantDateFV'].isna().all()
    
    for _, row in time_filtered.iterrows():
        grantDateFV = row['grantDateFV']
        sDate = pd.to_datetime(row['startDate'])
        eDate = pd.to_datetime(row['endDate'])
        gDate = pd.to_datetime(row['grantDate'])
        oops = False
        if sDate == eDate:
            eDate += pd.Timedelta(days=10)
            oops = True
        
        if pd.notna(grantDateFV):
            height = max(20 * grantDateFV / max_value, 1)
            # Create a rectangle for base salary
            rect = patches.Rectangle(
                (sDate, current_y),  # (x, y) position
                eDate - sDate,  # width
                height,  # height in data units
                color='#E9C46A',
                label='Time-based'
            )
            
            ax.add_patch(rect)  # Add rectangle to the plot
            if oops == True:
                ax.scatter(gDate, current_y+height/2, color='black', marker='o', s=6, label="Grant Date" if not legend_labels['Grant Date'] else "", zorder=0)
            else:
                ax.scatter(gDate, current_y+height/2, color='black', marker='o', s=6, label="Grant Date" if not legend_labels['Grant Date'] else "", zorder=5)
            legend_labels['Time-based'] = True
            legend_labels['Grant Date'] = True

            current_y += height+2
        else:
            ax.hlines(
                y=current_y,
                xmin=sDate,
                xmax=eDate,
                color="#E9C46A",
                linestyle="dashed",
                linewidth=1.5,  # Adjust thickness of dashed line
                label='Time-based' if all_nan else ""
            )
            if oops == True:
                ax.scatter(gDate, current_y, color='black', marker='o', s=6, label="Grant Date" if not legend_labels['Grant Date'] else "", zorder=0)
            else:
                ax.scatter(gDate, current_y, color='black', marker='o', s=6, label="Grant Date" if not legend_labels['Grant Date'] else "", zorder=5)
            legend_labels['Time-based'] = True
            legend_labels['Grant Date'] = True
            current_y += 2

    current_y += 2
    time_y = (current_y + time_y)/2
        

    # --- Plot Performance-Based ---
    sorted_grant_ids = sorted(grouped.groups.keys(), key=lambda x: group_grant_dates[x])

    for grant_id in sorted_grant_ids:
        performance_grouping = grant.loc[grant['grantID'] == grant_id, 'performanceGrouping']
    
        # If grant_id is found, get the first matching value
        performance_grouping = performance_grouping.iloc[0] if not performance_grouping.empty else None
        
        ax.axhline(y=current_y, color='gray', linestyle='dashed', alpha=0.3)
        # old_y = current_y
        group_df = grouped.get_group(grant_id)  # Retrieve the actual DataFrame
        label_for_group = (f"grant {grant_id}" if pd.notna(grant_id) else "unknown grant") 
        group_df = group_df.copy()
        group_df['metric_absrel'] = group_df['metric']+group_df['absRel']

        novalue = False
        if group_df['value'].isna().all():
            novalue = True
        else:
            first_value = group_df['value'].iloc[0] if not group_df['value'].isna().all() else min_value
            total_height = max(20 * (first_value / max_value) if max_value else 20,1)
            # 1) Check if percentVest has NaNs
            if group_df['percentVest'].isna().any():
                use_count_based = True
                total_count = len(group_df)
                group_df['computedVest'] = total_height / total_count  # Equal height for all rows
            else:
                use_count_based = False
                total_percentVest = group_df['percentVest'].sum()
                if total_percentVest == 0 or pd.isna(total_percentVest):
                    total_percentVest = 1  # Avoid division by zero
                group_df['computedVest'] = np.maximum(group_df['percentVest'] / total_percentVest * total_height, 0.8)
        
        # Identify distinct combos within this group
        # Step 1: Create a unique schedule for each metric
        # Convert schedule into a tuple set per metric
        schedule_dict = {}
        for metric, group in group_df.groupby('metric_absrel'):
            unique_schedule = group[['grantDate', 'startDate', 'endDate']].drop_duplicates()
            schedule = tuple(unique_schedule.apply(tuple, axis=1))
            schedule_dict[metric] = schedule
        
        # Step 2: Group metrics by identical schedules
        schedule_to_metrics = {}
        for metric, schedule in schedule_dict.items():
            if schedule in schedule_to_metrics:
                schedule_to_metrics[schedule].append(metric)
            else:
                schedule_to_metrics[schedule] = [metric]
        
        # Step 3: Assign group labels to metrics
        metric_to_group = {}
        for group_idx, (schedule, metrics) in enumerate(schedule_to_metrics.items(), start=1):
            for metric in metrics:
                metric_to_group[metric] = f"Group_{group_idx}"
        
        # Step 4: Add group information back to the original DataFrame
        group_df = group_df.copy()
        group_df['metric_group'] = group_df['metric_absrel'].map(metric_to_group)
        # print(group_df)

        further_groups = group_df.groupby(['metric_group'], dropna=False)
        
        # If only 1 distinct timeline, we label the y-axis with all metrics from the entire group
        # else, we label each timeline line with its metrics
        if len(further_groups) == 1:
            
            # print(grant_id,1)
            # There's just one combo => we can directly get that
            metric_group, metric_df = list(further_groups)[0]
            combo_groups = metric_df.groupby(['grantDate', 'startDate', 'endDate'], dropna=False)

            sorted_metrics = list(set(metric_df['metric'].tolist()[::-1]))
            metric_label = ", ".join(sorted_metrics) if sorted_metrics else "No Metric"

            grant_intro += f'Grant ID: {grant_id}\nGrouping: {performance_grouping}\nMetrics: {metric_label}\n'

            current_y += 6
            base_y = current_y
            
            for (gDate, sDate, eDate), sub_df in combo_groups:
                # Plot one line
                if novalue:
                    # print(grant_id,'novalue')
                    color = color_map.get(row['absRel'], 'gray')
                    ax.hlines(
                        y=current_y,
                        xmin=sDate,
                        xmax=eDate,
                        color=color,
                        linestyle="dashed",
                        linewidth=1.5  # Adjust thickness of dashed line
                    )
                    ax.scatter(gDate, current_y, color='black', marker='o', s=20, label="Grant Date" if not legend_labels['Grant Date'] else "", zorder=5)
                    legend_labels['Grant Date'] = True
                else:
                    sub_df = sub_df.sort_values(by=['absRel', 'computedVest'], ascending=[False, True])
                    # 3) Compute the vertical stacking based on computedVest
                    start_y = current_y
                    for _, row in sub_df.iterrows():
                        height = row['computedVest']
                        color = color_map.get(row['absRel'], 'gray')
        
                        line_label = 'absolute, performance period' if row['absRel'] == 'abs' else 'relative, performance period'

                        # print(grant_id,metric,sDate,eDate,start_y)
                        # Create a rectangle patch instead of using `ax.plot`
                        rect = patches.Rectangle(
                            (sDate, start_y),  # (x, y) position
                            eDate - sDate,  # width
                            height,  # height in data units
                            facecolor=color,
                            edgecolor=None,  # Optional: add border for better visibility
                            linewidth=0.2,  # Keep a thin border
                            label=line_label if not legend_labels[line_label] else None
                        )
                        
                        ax.add_patch(rect)  # Add rectangle to the plot
        
                        # Draw a dashed line between stacked rectangles (except for the last one)
                        if _ != sub_df.index[-1]:  
                            ax.hlines(
                                y=start_y + height,  # Position at top of current rectangle
                                xmin=sDate,
                                xmax=eDate,
                                color="white",
                                linestyle="dashed",
                                linewidth=0.5  # Adjust thickness of dashed line
                            )
        
                        
                        legend_labels[line_label] = True  # Mark legend entry as used
                
                        # Move up to stack the next segment
                        start_y += height  # Ensure consistent stacking in data units
                
                    # Plot the grant date marker
                    ax.scatter(gDate, (current_y + start_y) / 2, color='black', marker='o', s=6, label="Grant Date" if not legend_labels['Grant Date'] else "", zorder=5)
                    legend_labels['Grant Date'] = True
                    current_y = start_y+3
    
            # We'll label the entire group's y-tick with the list of metrics
            y_positions.append((base_y+ current_y)/2)
            y_labels.append(f"{label_for_group}")

            current_y += 6
        
        else:
            # print(grant_id,0)
            # Multiple distinct combos => each timeline gets its own line
            # We'll label each line with the relevant metrics
            lines_in_group = 0
            current_y += 6
            count = 0
            grant_intro += f'Grant ID: {grant_id}\nGrouping: {performance_grouping}\n'
            
            for metric_group, metric_df in further_groups:
                base_y = current_y
                count += 1
                if count >1:
                    current_y += 6
                    ax.axhline(y=current_y, color='gray', linestyle='dashed', alpha=0.3)
                    current_y += 6
                base_y = current_y
                    
                combo_groups = metric_df.groupby(['grantDate', 'startDate', 'endDate'], dropna=False)

                sorted_metrics = list(set(metric_df['metric'].tolist()[::-1]))
                
                metric_label = ", ".join(sorted_metrics) if sorted_metrics else "No Metric"
    
                grant_intro += f'  {grant_id}({count}) metrics: {metric_label}\n'
    
                
                for (gDate, sDate, eDate), sub_df in combo_groups:
                    # Plot one line
                    if novalue:
                        color = color_map.get(row['absRel'], 'gray')
                        ax.hlines(
                            y=current_y,
                            xmin=sDate,
                            xmax=eDate,
                            color=color,
                            linestyle="dashed",
                            linewidth=1.5  # Adjust thickness of dashed line
                        )
                        ax.scatter(gDate, current_y, color='black', marker='o', s=20, label="Grant Date" if not legend_labels['Grant Date'] else "", zorder=5)
                        legend_labels['Grant Date'] = True
                    else:
                        sub_df = sub_df.sort_values(by=['absRel', 'computedVest'], ascending=[False, True])
                        # 3) Compute the vertical stacking based on computedVest
                        start_y = current_y
                        for _, row in sub_df.iterrows():
                            height = row['computedVest']
                            color = color_map.get(row['absRel'], 'gray')
            
                            line_label = 'absolute, performance period' if row['absRel'] == 'abs' else 'relative, performance period'
            
                            # Create a rectangle patch instead of using `ax.plot'
                            rect = patches.Rectangle(
                                (sDate, start_y),  # (x, y) position
                                eDate - sDate,  # width
                                height,  # height in data units
                                facecolor=color,
                                edgecolor=None,  # Optional: add border for better visibility
                                linewidth=0.2,  # Keep a thin border
                                label=line_label if not legend_labels[line_label] else None
                            )
                            
                            ax.add_patch(rect)  # Add rectangle to the plot
            
                            # Draw a dashed line between stacked rectangles (except for the last one)
                            if _ != sub_df.index[-1]:
                                ax.hlines(
                                    y=start_y + height,  # Position at top of current rectangle
                                    xmin=sDate,
                                    xmax=eDate,
                                    color="white",
                                    linestyle="dashed",
                                    linewidth=0.5  # Adjust thickness of dashed line
                                )
            
                            
                            legend_labels[line_label] = True  # Mark legend entry as used
                    
                            # Move up to stack the next segment
                            start_y += height  # Ensure consistent stacking in data units
                    
                        # Plot the grant date marker
                        ax.scatter(gDate, (current_y + start_y) / 2, color='black', marker='o', s=6, label="Grant Date" if not legend_labels['Grant Date'] else "", zorder=5)
                        legend_labels['Grant Date'] = True
                        current_y = start_y+3
                        
                # We'll label the entire group's y-tick with the list of metrics
                y_positions.append((base_y+ current_y)/2)
                y_labels.append(f"{label_for_group}({count})")
    
            current_y += 6

    # Decorate axes
    ax.set_xlabel("Time")
    ax.set_ylabel("Grant-level grouping")

    # Format x-axis as Year-Month
    ax.xaxis.set_major_locator(mdates.YearLocator())
    ax.xaxis.set_minor_locator(mdates.MonthLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    plt.xticks(rotation=45)

    # y-ticks
    ax.set_yticks(y_positions + [time_y, option_y, stock_y, base_salary_y])
    ax.set_yticklabels(y_labels + ["Time-Based Awards", "Option Awards", "Stock Awards", "Base Salary"])

    ax.grid(axis='x', linestyle='--', alpha=0.6)

    # --- Ensure Proper Legend ---
    handles, labels = ax.get_legend_handles_labels()
    desired_order = ['absolute, performance period', 'relative, performance period', 'Grant Date', 'Non-performance-based', 'Time-based']
    final_labels = [lbl for lbl in desired_order if legend_labels.get(lbl, False)]
    ordered_handles = [handles[labels.index(lbl)] for lbl in final_labels if lbl in labels]
    ax.legend(ordered_handles, final_labels, loc='upper left')

    plt.tight_layout()
    plt.title(f"Grant-Level Timeline for {df_combined['companyName'].iloc[0]}, CEO {participantid}")
    plt.show()
    print(grant_intro)

In [57]:
# Build sets of all names, all CIKs:
all_company_names = sorted({nm for (_, nm, _) in unique_list})
all_ciks = sorted({c for (c, _, _) in unique_list})
all_ceos = sorted({p for (_, _, p) in unique_list})
# Create an Output widget to capture logs
debug_output = widgets.Output()

def filter_company_names(substring):
    """Return a list of company names containing 'substring' (case-insensitive)."""
    s_lower = substring.lower()
    return [n for n in all_company_names if s_lower in n.lower()]

def filter_ceos(substring):
    """Return a list of CEOs (as strings) containing 'substring'."""
    return [str(p) for p in all_ceos if substring in str(p)]

def names_for_ceo(ceo_name):
    """All company names that match the given CIK in company_list."""
    try:
        return sorted({
            n for (n, ceo) in unique_ceo if ceo == ceo_name
        })
    except ValueError:
        return []

def ceos_for_name(company_name):
    """All CEO names that match the given CIK in ceo_list."""
    try:
        matched_ceos = sorted({
            ceo for (n, ceo) in unique_ceo if n == company_name
        })
        return matched_ceos
    except ValueError:
        with debug_output:
            print(f"Error: Invalid company_name.")  # Debugging log
        return []  # Ensure function does not break

def on_company_typed(change):
    if change['name'] == 'value':
        typed = change['new']
        # Filter company suggestions
        company_widget.options = [
            n for n in all_company_names 
            if typed.lower() in n.lower()
        ]
        
        if typed in all_company_names:
            matched_ceos = ceos_for_name(typed)
            matched_ceos = [str(ceo) for ceo in matched_ceos]
            participant_widget.options = matched_ceos

            # If the current typed CIK isn't in matched_ciks, clear the CIK
            if participant_widget.value not in matched_ceos:
                participant_widget.value = ""  # Clear if mismatch
        else:
            # Not a finalized name => you could restore full CIK list or partial filter
            participant_widget.value = ""
            participant_widget.options = all_ceos
            pass

def on_ceo_typed(change):
    if change['name'] == 'value':
        typed = change['new'].strip()
        participant_widget.options = filter_ceos(typed)
        known_ceos = [str(p) for p in all_ceos]
        
        # If typed is exactly one known CIK:
        if typed in known_ciks:
            # Show only the matched company name(s)
            matched_names = names_for_ceo(typed)
            company_widget.options = matched_names

            # If the current typed company isn't in matched_names, clear the company
            if company_widget.value not in matched_names:
                company_widget.value = ""
        else:
            # Not a finalized CIK => you could restore full name list or partial filter
            company_widget.value = ""
            company_widget.options = all_company_names

def on_run_button_click(b):
    """
    Clears the output area, then runs the plotting function.
    """
    with output_area:
        output_area.clear_output()
        
        company_val = company_widget.value
        participant_val = participant_widget.value

        # Now call your timeline function(s)
        plot_compensation_timeline_by_grant(company_val, participant_val, False)

# --- Create dynamic Comboboxes for Company & CIK ---
company_widget = widgets.Combobox(
    placeholder='Type a company name...',
    options=all_company_names,
    description='Company:'
)
company_widget.value = ""  # your existing default

participant_widget = widgets.Combobox(
    placeholder='Type a CEO ID...',
    options=[str(p) for p in all_ceos],
    description='CEO ID:'
)
participant_widget.value = ""  # your existing default

run_button = widgets.Button(description="Generate Timeline")
output_area = widgets.Output()

company_widget.observe(on_company_typed, names='value')
participant_widget.observe(on_ceo_typed, names='value')
# Create a title widget
title_widget = widgets.HTML(
    value="<h1 style='text-align: center;'>Visualizing CEO Compensation</h1>"
)

run_button.on_click(on_run_button_click)

# Create the Clear button
clear_button = widgets.Button(description="Clear", button_style='warning')

# Function to clear input fields
def on_clear_button_click(b):
    company_widget.value = ""
    participant_widget.value = ""

# Attach event to Clear button
clear_button.on_click(on_clear_button_click)

# Display the widgets & output in the notebook
display(
    widgets.VBox([
        title_widget,  # Add the title at the top
        widgets.HBox([company_widget, participant_widget]),
        widgets.HBox([run_button, clear_button]),
        output_area
    ])
)
display(debug_output)

Output()